In [68]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score 
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score 
import optuna
import numpy as np 
import os
from sklearn import metrics
import matplotlib.pyplot as plt

In [69]:
# 估計值
def Score(m, x_train, y_train, x_test, y_test, train=True):
    # training 的
    if train:
        pred=m.predict(x_train)
        print('Train Result:\n')
        print(f"Accuracy Score: {accuracy_score(y_train, pred)*100:.2f}%")
        print(f"Precision Score: {precision_score(y_train, pred)*100:.2f}%")
        print(f"Recall Score: {recall_score(y_train, pred)*100:.2f}%")
        print(f"F1 score: {f1_score(y_train, pred)*100:.2f}%")
        print(f"Confusion Matrix:\n {confusion_matrix(y_train, pred)}")
    # testing 的
    elif train == False:
        pred=m.predict(x_test)
        print('Test Result:\n')
        print(f"Accuracy Score: {accuracy_score(y_test, pred)*100:.2f}%")
        print(f"Precision Score: {precision_score(y_test, pred)*100:.2f}%")
        print(f"Recall Score: {recall_score(y_test, pred)*100:.2f}%")
        print(f"F1 score: {f1_score(y_test, pred)*100:.2f}%")
        print(f"Confusion Matrix:\n {confusion_matrix(y_test, pred)}")
        

In [70]:
# 讀data
state = "ask"
FILE_PATH = rf"C:\Users\yicheng\Desktop\high-frequency-trading-MCTS\stock_dataset_with_label\2330\{state}\*"


files = glob.glob(FILE_PATH)
data=[]
data = pd.DataFrame(data)

for f in files:
    
    d = pd.read_csv(f, index_col=None)
    d = d.drop(columns=["matchPri",'bidPri1','bidPri2','bidPri3','bidPri4','bidPri5','askPri1','askPri2','askPri3','askPri4','askPri5',"openPri"])
    data = pd.concat([data,d]) 
    

print(data.shape)

(527175, 12)


In [71]:
# Training and Validation  
# we split data to 10 parts
# TV1 means split data to training 1 part ,validation 9 parts
TV = 1

In [72]:
# the length of total data 
length = len(data)
# the length of training data 
train_length = int(length * TV *0.1)
# split training data
train_data = data[:train_length]
# train input data 
X_train = train_data.drop(columns=['label'])
# train label 
y_train = train_data['label'].to_numpy()

# split testing data
test_data = data[train_length:]
# tets input data 
X_test = test_data.drop(columns=['label'])
# test label
y_test = test_data['label'].to_numpy()



In [ ]:
print('特徵重要程度: ',model.feature_importances_)

特徵重要程度:  [0.09583836 0.07846119 0.08632863 0.09179962 0.0960384  0.09726837
 0.07891914 0.08856925 0.09223704 0.09600724 0.09853285]


In [74]:
print(f"Train : {len(X_train)} {len(y_train)}")
print(f"Test  : {len(X_test)} {len(y_test)}")

Train : 52717 52717
Test  : 474458 474458


In [75]:
# 計算training 的 label 比例分配
change = 0
other = 0
for i in y_train:
    
    if i == 0:
        other+=1
    else:
        change+=1

print(change,other)
print('change: {:%}'.format(change/(change+other)))
print('other : {:%}'.format(other/(change+other)))

8397 44320
change: 15.928448%
other : 84.071552%


In [76]:
best_recall = -1

In [78]:
# run weight 
def Objective(trial):

    global best_recall
    # 配置要選的權重和range
    # 'scale_pos_weight':other/down,這個一定要加 ，他是處理imbalance的 
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0,log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50,500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0,log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0,log=True),
        'scale_pos_weight':other/change,
        'reg_alpha':  trial.suggest_float('reg_lambda', 1e-8, 1.0,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0,log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1.0),
        # CPU thread 
        'nthread':16
    }

    # 訓練
    model = xgb.XGBClassifier(**param)  
    
    model.fit(X_train, y_train,verbose=False)
    # 預測
    X_pred = model.predict(X_train)
    # 評估分數
    recall = round(f1_score(y_train, X_pred),2)
    if recall > best_recall:
        # 存model weight
        model.save_model(f'xgb_best_{state}{recall}.model')

        if os.path.isfile(f'./xgb_best_{state}{best_recall}.model'):
            os.remove(f'./xgb_best_{state}{best_recall}.model')
        
        best_recall = recall
        
    return recall

In [79]:
# 找參數的套件 ㄝㄝ, 很佔資源
# direction ="maximize" ,代表作為評估的值要找最大值
study = optuna.create_study(direction='maximize')

# pre-train 用
# study.enqueue_trial(study.best_trial)

# n_trials 要跑幾次
study.optimize(Objective, n_trials = 10,show_progress_bar=True)

[I 2023-03-20 23:49:51,545] A new study created in memory with name: no-name-825e73df-c123-4c0d-91df-d84c5f13742e
c:\Users\yicheng\anaconda3\envs\gpu_env\lib\site-packages\optuna\progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-03-20 23:49:54,086] Trial 0 finished with value: 0.48 and parameters: {'max_depth': 11, 'subsample': 0.4707688609026088, 'n_estimators': 140, 'learning_rate': 0.09143993852417222, 'gamma': 1.2514208815765042e-06, 'reg_lambda': 4.625787676464828e-08, 'min_child_weight': 8, 'colsample_bytree': 0.42050494615712947}. Best is trial 0 with value: 0.48.
[I 2023-03-20 23:50:01,525] Trial 1 finished with value: 0.57 and parameters: {'max_depth': 10, 'subsample': 0.7350646968808751, 'n_estimators': 411, 'learning_rate': 0.10100765615720714, 'gamma': 0.00021075357935703697, 'reg_lambda': 0.009050017965259411, 'min_child_weight': 17, 'colsample_bytree': 0.8421060575085494}. Best is trial 1 with value: 0.57.
[I 2023-03-20 23:50:02,509] Trial 2 finished with value: 0.29 and parameters: {'max_depth': 7, 'subsample': 0.022541702615069808, 'n_estimators': 100, 'learning_rate': 0.18485925899363448, 'gamma': 1.1808883534452803e-07, 'reg_lambda': 9.402829472295817e-08, 'min_child_weight': 8, 'cols

In [88]:
# 輸出找到的最佳參數
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 10
Best trial:
  Value: 0.76
  Params: 
    max_depth: 15
    subsample: 0.6563881849944677
    n_estimators: 163
    learning_rate: 0.1570500000166422
    gamma: 1.0676114091532102e-08
    reg_lambda: 0.0929132902748759
    min_child_weight: 4
    colsample_bytree: 0.666550060178037


In [95]:
# 看一下效果
model = xgb.XGBClassifier()

# 放best model
model_path = rf'C:\Users\yicheng\Desktop\high-frequency-trading-MCTS\model\xgb_best_ask0.76.model'

model.load_model(model_path)
# training eval
Score(model,X_train, y_train, X_test,y_test)
# testing eval
Score(model,X_train, y_train, X_test,y_test,train=False)

print('特徵重要程度: ',model.feature_importances_)

C:\Users\yicheng\Desktop\high-frequency-trading-MCTS\model\xgb_best_ask0.76.model
Train Result:

Accuracy Score: 90.25%
Precision Score: 62.18%
Recall Score: 99.00%
F1 score: 76.38%
Confusion Matrix:
 [[39263  5057]
 [   84  8313]]
Test Result:

Accuracy Score: 75.18%
Precision Score: 17.98%
Recall Score: 12.97%
F1 score: 15.07%
Confusion Matrix:
 [[346274  47661]
 [ 70078  10445]]
特徵重要程度:  [0.10473517 0.08821649 0.08229707 0.09298414 0.09291208 0.08847114
 0.08723172 0.08552441 0.08176731 0.0955264  0.10033416]


In [96]:
# 這邊就不斷調要train 幾次 ,看recall score 
# **trial.params : 前面找出來的參數
xgb_model = xgb.XGBClassifier(**trial.params)

# pre train 的次數
num =10 

# model_path = f'.\model\xgb_best_{state}{round(trial,2)}.model'
for i in range(num):
    # xgb_model 讀檔的
    xgb_model.fit(X_train, y_train,verbose=1,xgb_model=model_path)
    # 存檔
    xgb_model.save_model(model_path)


Score(xgb_model,X_train, y_train, X_test,y_test)
Score(xgb_model,X_train, y_train, X_test,y_test,train=False)

Train Result:

Accuracy Score: 99.25%
Precision Score: 95.52%
Recall Score: 100.00%
F1 score: 97.71%
Confusion Matrix:
 [[43926   394]
 [    0  8397]]
Test Result:

Accuracy Score: 78.26%
Precision Score: 17.63%
Recall Score: 7.65%
F1 score: 10.67%
Confusion Matrix:
 [[365139  28796]
 [ 74360   6163]]
